In [1]:
!pip install tensorflow-gpu==1.14.0
!pip install keras==2.2.4
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()

     |████████████████████████████████| 377.0MB 31kB/s 
     |████████████████████████████████| 3.2MB 27.5MB/s 
     |████████████████████████████████| 491kB 41.9MB/s 
ERROR: tensorflow 2.0.0 has requirement tensorboard<2.1.0,>=2.0.0, but you'll have tensorboard 1.14.0 which is incompatible.
ERROR: tensorflow 2.0.0 has requirement tensorflow-estimator<2.1.0,>=2.0.0, but you'll have tensorflow-estimator 1.14.0 which is incompatible.
  Found existing installation: tensorboard 2.0.0
    Uninstalling tensorboard-2.0.0:
      Successfully uninstalled tensorboard-2.0.0
  Found existing installation: tensorflow-estimator 2.0.1
    Uninstalling tensorflow-estimator-2.0.1:
      Successfully uninstalled tensorflow-estimator-2.0.1
     |████████████████████████████████| 317kB 2.8MB/s 
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/glove840b300dtxt/glove.840B.300d.txt
/kaggle/input/applications-of-deep-learningwustl-fall-2019/train.csv
/kaggle/input/applications-of-deep-learningwustl-fall-2019/sample.csv
/kaggle/input/applications-of-deep-learningwustl-fall-2019/test.csv
/kaggle/input/quora-question-pairs/train.csv
/kaggle/input/quora-question-pairs/sample_submission.csv
/kaggle/input/quora-question-pairs/test.csv


In [3]:
import re, collections
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, Dropout, add, concatenate
from keras.layers import LSTM, CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.layers.normalization import BatchNormalization
import keras.backend as K
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [4]:
def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stop_words = set(stopwords.words("english"))
        text = [w for w in text if not w in stop_words]
    
    text = " ".join(text)
    
    # Remove punctuation from text
    # text = "".join([c for c in text if c not in punctuation])

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    # text = re.sub(r"\0s", "0", text) # It doesn't make sense to me
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)


In [5]:
EMBEDDING_FILES = [
    #'../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec',
    '../input/glove840b300dtxt/glove.840B.300d.txt'
]
NUM_MODELS = 2
MAX_FEATURES = 1000000
BATCH_SIZE = 512
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
EPOCHS = 4
MAX_LEN = 60

In [6]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')


def load_embeddings(path):
    with open(path) as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in f)


def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            pass
    return embedding_matrix

In [7]:


    

def build_model(embedding_matrix):
    words1 = Input(shape=(MAX_LEN,))
    words2 = Input(shape=(MAX_LEN,))
    magic_input = Input(shape=(train_leaks.shape[1],))
    magic_dense = Dense(64, activation='relu')(magic_input)
    x1 = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words1)
    x2 = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words2)
    x1 = SpatialDropout1D(0.25)(x1)
    x2 = SpatialDropout1D(0.25)(x2)
    x1 = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x1)
    x2 = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x2)
    x1 = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=False))(x1)
    x2 = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=False))(x2)
    #x1 = CuDNNLSTM(LSTM_UNITS)(x1)
    #x2 = CuDNNLSTM(LSTM_UNITS)(x2)
    
    hidden = concatenate([x1, x2, magic_dense])
    hidden = BatchNormalization()(hidden)
    hidden = Dropout(0.25)(hidden)
    hidden = Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)
    hidden = BatchNormalization()(hidden)
    hidden = Dropout(0.2)(hidden)
    result = Dense(1, activation='sigmoid')(hidden)
    
    model = Model(inputs=[words1, words2, magic_input], outputs=result)
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(clipnorm=0.1),
        metrics=['accuracy'])

    return model



In [8]:
train = pd.read_csv('../input/applications-of-deep-learningwustl-fall-2019/train.csv')
test = pd.read_csv('../input/applications-of-deep-learningwustl-fall-2019/test.csv')

In [9]:
train_ex = pd.read_csv("../input/quora-question-pairs/train.csv")

In [10]:
train_ex = train_ex[['id', 'question1', 'question2', 'is_duplicate']]
train_ex = train_ex.rename(columns={"question1": "sent1", "question2": "sent2", "is_duplicate": "same_source"})

In [11]:
train = train.append(train_ex, ignore_index=True)

In [12]:
s1_train = train['sent1'].fillna('').values
s2_train = train['sent2'].fillna('').values
y_train = train['same_source']
s1_test = test['sent1'].fillna('').values
s2_test = test['sent1'].fillna('').values

x1_train, x2_train = [], []
for line in s1_train:
    x1_train.append(text_to_wordlist(line, remove_stopwords=False, stem_words=False))
for line in s2_train:
    x2_train.append(text_to_wordlist(line, remove_stopwords=False, stem_words=False))
    
x1_test, x2_test = [], []
for line in s1_test:
    x1_test.append(text_to_wordlist(line, remove_stopwords=False, stem_words=False))
for line in s2_test:
    x2_test.append(text_to_wordlist(line, remove_stopwords=False, stem_words=False))

In [13]:
tokenizer = text.Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(x1_train + x1_test + x2_train + x2_test)

x1_train = tokenizer.texts_to_sequences(x1_train)
x1_test = tokenizer.texts_to_sequences(x1_test)
x2_train = tokenizer.texts_to_sequences(x2_train)
x2_test = tokenizer.texts_to_sequences(x2_test)

x1_train = sequence.pad_sequences(x1_train, maxlen=MAX_LEN)
x1_test = sequence.pad_sequences(x1_test, maxlen=MAX_LEN)
x2_train = sequence.pad_sequences(x2_train, maxlen=MAX_LEN)
x2_test = sequence.pad_sequences(x2_test, maxlen=MAX_LEN)

In [14]:
sentences = pd.concat([train[['sent1', 'sent2']], \
        test[['sent1', 'sent2']]], axis=0).reset_index(drop='index')
s_dict = collections.defaultdict(set)
for i in range(sentences.shape[0]):
        s_dict[sentences.sent1[i]].add(sentences.sent2[i])
        s_dict[sentences.sent2[i]].add(sentences.sent1[i])

def s1_freq(row):
    return(len(s_dict[row['sent1']]))
    
def s2_freq(row):
    return(len(s_dict[row['sent2']]))
    
def s1_s2_intersect(row):
    return(len(set(s_dict[row['sent1']]).intersection(set(s_dict[row['sent2']]))))

train['s1_s2_intersect'] = train.apply(s1_s2_intersect, axis=1, raw=True)
train['s1_freq'] = train.apply(s1_freq, axis=1, raw=True)
train['s2_freq'] = train.apply(s2_freq, axis=1, raw=True)

test['s1_s2_intersect'] = test.apply(s1_s2_intersect, axis=1, raw=True)
test['s1_freq'] = test.apply(s1_freq, axis=1, raw=True)
test['s2_freq'] = test.apply(s2_freq, axis=1, raw=True)

train_leaks = train[['s1_s2_intersect', 's1_freq', 's2_freq']]
test_leaks = test[['s1_s2_intersect', 's1_freq', 's2_freq']]

ss = StandardScaler()
ss.fit(np.vstack((train_leaks, test_leaks)))
train_leaks = ss.transform(train_leaks)
test_leaks = ss.transform(test_leaks)

In [15]:
embedding_matrix = np.concatenate(
    [build_matrix(tokenizer.word_index, f) for f in EMBEDDING_FILES], axis=-1)
    

In [16]:

checkpoint_predictions = []
weights = []

#for model_idx in range(NUM_MODELS):
model = build_model(embedding_matrix)
    #for global_epoch in range(EPOCHS):
model.fit(
    [x1_train, x2_train, train_leaks],
    y_train,
    validation_split = 0.15,
    shuffle = True,
    batch_size=BATCH_SIZE,
    epochs=50,
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=10)
        #LearningRateScheduler(lambda epoch: 1e-3 * (0.6 ** global_epoch))
    ]
)
        #checkpoint_predictions.append(model.predict([x1_test, x2_test], batch_size=2048)[0].flatten())
        #weights.append(2 ** global_epoch)

#predictions = np.average(checkpoint_predictions, weights=weights, axis=0)
predictions = model.predict([x1_test, x2_test, test_leaks], batch_size=2048) + model.predict([x2_test, x1_test, test_leaks], batch_size=2048)
predictions /= 2
predictions = predictions.ravel()



Train on 453429 samples, validate on 80017 samples
Epoch 1/50
453429/453429 [==============================] - 106s 233us/step - loss: 0.3332 - acc: 0.8600 - val_loss: 0.3544 - val_acc: 0.8476
Epoch 2/50
453429/453429 [==============================] - 99s 218us/step - loss: 0.2869 - acc: 0.8785 - val_loss: 0.3342 - val_acc: 0.8560
Epoch 3/50
453429/453429 [==============================] - 99s 218us/step - loss: 0.2697 - acc: 0.8840 - val_loss: 0.3119 - val_acc: 0.8600
Epoch 4/50
453429/453429 [==============================] - 99s 218us/step - loss: 0.2565 - acc: 0.8893 - val_loss: 0.3059 - val_acc: 0.8631
Epoch 5/50
453429/453429 [==============================] - 99s 218us/step - loss: 0.2465 - acc: 0.8930 - val_loss: 0.3048 - val_acc: 0.8635
Epoch 6/50
453429/453429 [==============================] - 99s 218us/step - loss: 0.2376 - acc: 0.8967 - val_loss: 0.2957 - val_acc: 0.8693
Epoch 7/50
453429/453429 [==============================] - 99s 218us/step - loss: 0.2284 - acc: 0.900

In [17]:
submission = pd.DataFrame.from_dict({
    'id': test['id'],
    'same_source': predictions
})

submission.to_csv('submission.csv', index=False)

In [18]:
submission.same_source.describe()

count    14350.000000
mean         0.510810
std          0.480405
min          0.000000
25%          0.000016
50%          0.641065
75%          0.998141
max          1.000000
Name: same_source, dtype: float64